# BEE 4750 Lab 2: Uncertainty and Monte Carlo

**Name**:

**ID**:

> **Due Date**
>
> Wednesday, 9/25/24, 9:00pm

## Setup

The following code should go at the top of most Julia scripts; it will
load the local package environment and install any needed packages. You
will see this often and shouldn’t need to touch it.

In [1]:
import Pkg
Pkg.activate(".")
Pkg.instantiate()

In [1]:
using Random # random number generation
using Distributions # probability distributions and interface
using Statistics # basic statistical functions, including mean
using Plots # plotting

## Overview

In this lab, we will use Monte Carlo analysis to analyze a version of
the culmination of every episode of the long-running game show [The
Price Is Right](https://en.wikipedia.org/wiki/The_Price_Is_Right): the
[Showcase](https://en.wikipedia.org/wiki/The_Price_Is_Right#Showcase).

Monte Carlo methods involve the simulation of random numbers from
probability distributions. In an environmental context, we often
propagate these random numbers through some more complicated model and
then compute a resulting statistic which is relevant for assessing
performance or risk, such as an average outcome or a particular
quantile.

You should always start any computing with random numbers by setting a
“seed,” which controls the sequence of numbers which are generated
(since these are not *really* random, just “pseudorandom”). In Julia, we
do this with the `Random.seed!()` function.

In [1]:
Random.seed!(1)

TaskLocalRNG()

It doesn’t matter what seed you set, though different seeds might result
in slightly different values. But setting a seed means every time your
notebook is run, the answer will be the same.

> **Seeds and Reproducing Solutions**
>
> If you don’t re-run your code in the same order or if you re-run the
> same cell repeatedly, you will not get the same solution. If you’re
> working on a specific problem, you might want to re-use
> `Random.seed()` near any block of code you want to re-evaluate
> repeatedly.

### Probability Distributions and Julia

Julia provides a common interface for probability distributions with the
[`Distributions.jl`
package](https://juliastats.org/Distributions.jl/stable/). The basic
workflow for sampling from a distribution is:

1.  Set up the distribution. The specific syntax depends on the
    distribution and what parameters are required, but the general call
    is the similar. For a normal distribution or a uniform distribution,
    the syntax is

    ``` julia
    # you don't have to name this "normal_distribution"
    # μ is the mean and σ is the standard deviation
    normal_distribution = Normal(μ, σ)
    # a is the upper bound and b is the lower bound; these can be set to +Inf or -Inf for an unbounded distribution in one or both directions.
    uniform_distribution = Uniform(a, b)
    ```

    There are lots of both
    [univariate](https://juliastats.org/Distributions.jl/stable/univariate/#Index)
    and
    [multivariate](https://juliastats.org/Distributions.jl/stable/multivariate/)
    distributions, as well as the ability to create your own, but we
    won’t do anything too exotic here.

2.  Draw samples. This uses the `rand()` command (which, when used
    without a distribution, just samples uniformly from the interval
    $[0, 1]$.) For example, to sample from our normal distribution
    above:

    ``` julia
    # draw n samples
    rand(normal_distribution, n)
    ```

Putting this together, let’s say that we wanted to simulate 100
six-sided dice rolls. We could use a [Discrete Uniform
distribution](https://juliastats.org/Distributions.jl/stable/univariate/#Distributions.DiscreteUniform).

In [1]:
dice_dist = DiscreteUniform(1, 6) # can generate any integer between 1 and 6
dice_rolls = rand(dice_dist, 100) # simulate rolls

100-element Vector{Int64}:
 3
 6
 4
 6
 1
 1
 2
 3
 5
 5
 ⋮
 3
 3
 6
 5
 3
 4
 6
 3
 1

And then we can plot a histogram of these rolls:

In [1]:
histogram(dice_rolls, legend=:false, bins=6)
ylabel!("Count")
xlabel!("Dice Value")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path clip-path="url(#clip660)" d="M698.122 1794.89 L714.441 1794.89 L714.441 1798.82 L692.497 1798.82 L692.497 1794.89 Q695.159 1792.13 699.742 1787.5 Q704.348 1782.85 705.529 1781.51 Q707.774 1778.98 708.654 1777.25 Q709.557 1775.49 709.557 1773.8 Q709.557 1771.04 707.612 1769.31 Q705.691 1767.57 702.589 1767.57 Q700.39 1767.57 697.936 1768.34 Q695.506 1769.1 692.728 1770.65 L692.728 1765.93 Q695.552 1764.79 698.006 1764.21 Q700.459 1763.64 702.496 1763.64 Q707.867 1763.64 711.061 1766.32 Q714.256 1769.01 714.256 1773.5 Q714.256 1775.63 713.445 1777.55 Q712.658 1779.45 710.552 1782.04 Q709.973 1782.71 706.871 1785.93 Q703.77 1789.12 698.122 1794.89 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip660)" d="M1068.42 1780.19 Q1071.78 1780.9 1073.66 1783.17 Q1075.55 1785.44 1075.55 1788.77 Q1075.55 1793.89 1072.03 1796.69 Q1068.52 1799.49 1062.03 1799.49 Q1059.86 1799.49 1057.54 1799.05 Q1055.25 1798.64 1052.8 1797.78 L1052.8 1793.27 Q1054.74 1794.4 1057.06 1794.98 Q1059.37 1795.56 1061.9 1795.56 Q1066.29 1795.56 1068.59 1793.82 Q1070.9 1792.08 1070.9 1788.77 Q1070.9 1785.72 1068.75 1784.01 Q1066.62 1782.27 1062.8 1782.27 L1058.77 1782.27 L1058.77 1778.43 L1062.98 1778.43 Q1066.43 1778.43 1068.26 1777.06 Q1070.09 1775.67 1070.09 1773.08 Q1070.09 1770.42 1068.19 1769.01 Q1066.32 1767.57 1062.8 1767.57 Q1060.88 1767.57 1058.68 1767.99 Q1056.48 1768.4 1053.84 1769.28 L1053.84 1765.12 Q1056.5 1764.38 1058.82 1764.01 Q1061.16 1763.64 1063.22 1763.64 Q1068.54 1763.64 1071.64 1766.07 Q1074.74 1768.47 1074.74 1772.59 Q1074.74 1775.46 1073.1 1777.46 Q1071.46 1779.42 1068.42 1780.19 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /> <path clip-path="url(#clip660)" d="M1775.87 1764.26 L1794.23 1764.26 L1794.23 1768.2 L1780.15 1768.2 L1780.15 1776.67 Q1781.17 1776.32 1782.19 1776.16 Q1783.21 1775.97 1784.23 1775.97 Q1790.01 1775.97 1793.39 1779.15 Q1796.77 1782.32 1796.77 1787.73 Q1796.77 1793.31 1793.3 1796.41 Q1789.83 1799.49 1783.51 1799.49 Q1781.33 1799.49 1779.06 1799.12 Q1776.82 1798.75 1774.41 1798.01 L1774.41 1793.31 Q1776.49 1794.45 1778.72 1795 Q1780.94 1795.56 1783.41 1795.56 Q1787.42 1795.56 1789.76 1793.45 Q1792.1 1791.34 1792.1 1787.73 Q1792.1 1784.12 1789.76 1782.02 Q1787.42 1779.91 1783.41 1779.91 Q1781.54 1779.91 1779.66 1780.33 Q1777.81 1780.74 1775.87 1781.62 L1775.87 1764.26 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /><path clip-path="url(#clip660)" d="M2146.7 1779.68 Q2143.56 1779.68 2141.7 1781.83 Q2139.87 1783.98 2139.87 1787.73 Q2139.87 1791.46 2141.7 1793.64 Q2143.56 1795.79 2146.7 1795.79 Q2149.85 1795.79 2151.68 1793.64 Q2153.53 1791.46 2153.53 1787.73 Q2153.53 1783.98 2151.68 1781.83 Q2149.85 1779.68 2146.7 1779.68 M2155.99 1765.02 L2155.99 1769.28 Q2154.23 1768.45 2152.42 1768.01 Q2150.64 1767.57 2148.88 1767.57 Q2144.25 1767.57 2141.8 1770.7 Q2139.37 1773.82 2139.02 1780.14 Q2140.38 1778.13 2142.44 1777.06 Q2144.5 1775.97 2146.98 1775.97 Q2152.19 1775.97 2155.2 1779.15 Q2158.23 1782.29 2158.23 1787.73 Q2158.23 1793.06 2155.08 1796.27 Q2151.93 1799.49 2146.7 1799.49 Q2140.71 1799.49 2137.54 1794.91 Q2134.37 1790.3 2134.37 1781.58 Q2134.37 1773.38 2138.25 1768.52 Q2142.14 1763.64 2148.69 1763.64 Q2150.45 1763.64 2152.24 1763.98 Q2154.04 1764.33 2155.99 1765.02 Z" fill="#000000" fill-rule="nonzero" fill-opacity="1" /> <path clip-path="url(#clip660)" d="M1351.53 1849.02 L1351.53 1854.5 Q1349.05 1853.13 1346.54 1852.46 Q1344.05 1851.76 1341.51 1851.76 Q1335.81 1851.76 1332.66 1855.39 Q1329.51 1858.98 1329.51 1865.51 Q1329.51 1872.03 1332.66 1875.66 Q1335.81 1879.26 1341.51 1879.26 Q1344.05 1879.26 1346.54 1878.59 Q1349.05 1877.89 1351.53 1876.52 L1351.53 1881.93 Q1349.08 1883.08 1346.44 1883.65 Q1343.83 1884.22 1340.87 1884.22 Q1332.82 1884.22 1328.08 1879.16 Q1323.33 1874.1 1323.33 1865.51 Q1323.33 1856.79 1328.11 1851.79 Q1332.91 1846.79 1341.25 1846.79 Q13

### Instructions

**Remember to**:

-   Evaluate all of your code cells, in order (using a `Run All`
    command). This will make sure all output is visible and that the
    code cells were evaluated in the correct order.
-   Tag each of the problems when you submit to Gradescope; a 10%
    penalty will be deducted if this is not done.

## Exercise (3 Points)

The Showcase is the final round of every episode of The Price is Right,
matching the two big winners from the episode. Each contestant is shown
a “showcase” of prizes, which are usually some combination of a trip, a
motor vehicle, some furniture, and maybe some other stuff. They then
each have to make a bid on the retail price of the showcase. The rules
are:

-   an overbid is an automatic loss;
-   the contest who gets closest to the retail price wins their
    showcase;
-   if a contestant gets within \$250 of the retail price and is closer
    than their opponent, they win both showcases.

Your goal is to find a wager which maximizes your expected winnings,
which we may as well call utility, based on your assessment of the
probability of your showcase retail price. Make the following
assumptions about your expected winnings if you don’t overbid:

-   If you win both showcases, the value is the double of the single
    showcase value.
-   If you did not win both showcases but bid under the showcase value,
    the probability of being outbid increases linearly as the distance
    between your bid and the value increases (in other words, if you bid
    the exact value, you win with probability 1, and if you bid \$0, you
    win with probability 0).

We’ll assume that the distribution of all showcases offered by the show
is given as truncated normal distribution, which means a normal
distribution which has an upper and/or lower bound. `Distributions.jl`
makes it easy to specify truncations on any distribution, not just
normal distributions. For example, we’ll use the distribution shown in
<a href="#fig-values" class="quarto-xref">Figure 1</a> for the showcase
values (you can just directly use the `showcase_dist` distribution in
your lab code).

In [1]:
showcase_dist = truncated(Normal(31000, 4500), lower=5000, upper=42000)
showcase_samples = rand(showcase_dist, 10_000)
histogram(showcase_samples)

Find your expected winnings if you bid \$35,000. Plot the outcomes of
your Monte Carlo experiment (iterations vs. running estimate). How did
you decide how many samples to use?

## References

Put any consulted sources here, including classmates you worked with/who
helped you.